<a href="https://colab.research.google.com/github/adityasahusomu/Youtube_Comment_Analyzer/blob/main/exp_5_xgboost_with_HPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Till now, according to the experiments we have conducted,


* Feature engineering- Tf-idf

* ngram- trigram
* max_feature- 1000


* Handing unbalance data- Undersampling (SMOTE)


Next step: Experimenting with differnt algorithm and then HP tuning using Optuna (Bayesian Optimization)

---

MLflow Logging= 7 runs for 7 algos (using the best parameters through HPT)


In [1]:
!pip install mlflow boto3 awscli

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 32.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 80.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 68.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 119.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 88.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 570.5/570.5 kB 36.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 329.1/329.1 kB 24.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.2/8

In [2]:
import os, getpass

os.environ["AWS_ACCESS_KEY_ID"] = getpass.getpass("Enter AWS Access Key ID: ")
os.environ["AWS_SECRET_ACCESS_KEY"] = getpass.getpass("Enter AWS Secret Access Key: ")
os.environ["AWS_DEFAULT_REGION"] = "us-east-1"

Enter AWS Access Key ID: ··········
Enter AWS Secret Access Key: ··········


In [3]:
import boto3
s3 = boto3.client('s3')
print(s3.list_buckets())

{'ResponseMetadata': {'RequestId': 'MWX89X8SWNFWGRC4', 'HostId': 'Q7hVkDqQ24VUzD9jtrk6p3llC01DlanLV8MgJUcpV5NZ3fNQTb5I0WLi/tk3l2crtZRfeECbsww9HRy/gI686PWUiQI39A9e', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amz-id-2': 'Q7hVkDqQ24VUzD9jtrk6p3llC01DlanLV8MgJUcpV5NZ3fNQTb5I0WLi/tk3l2crtZRfeECbsww9HRy/gI686PWUiQI39A9e', 'x-amz-request-id': 'MWX89X8SWNFWGRC4', 'date': 'Thu, 16 Oct 2025 10:24:50 GMT', 'content-type': 'application/xml', 'transfer-encoding': 'chunked', 'server': 'AmazonS3'}, 'RetryAttempts': 0}, 'Buckets': [{'Name': 'project1-mlflow-bucket', 'CreationDate': datetime.datetime(2025, 10, 12, 12, 4, 13, tzinfo=tzlocal())}], 'Owner': {'DisplayName': 'adityasahusomu02', 'ID': '866466d6c4d8a9893e39cdce3c468d75133f23de2c907deeda4f1a78a7ca565b'}}


In [4]:
import mlflow
mlflow.set_tracking_uri("http://ec2-3-15-32-230.us-east-2.compute.amazonaws.com:5000/")

print("Tracking URI:", mlflow.get_tracking_uri())

Tracking URI: http://ec2-3-15-32-230.us-east-2.compute.amazonaws.com:5000/


In [6]:
mlflow.set_experiment("exp 5 -XgBoost with HP Tuning")

2025/10/16 10:25:46 INFO mlflow.tracking.fluent: Experiment with name 'exp 5 -XgBoost with HP Tuning' does not exist. Creating a new experiment.


<Experiment: artifact_location='s3://project1-mlflow-bucket/716770175548013518', creation_time=1760610346865, experiment_id='716770175548013518', last_update_time=1760610346865, lifecycle_stage='active', name='exp 5 -XgBoost with HP Tuning', tags={}>

In [7]:
!pip install optuna xgboost imbalanced-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.9/400.9 kB 5.9 MB/s eta 0:00:00


In [8]:
import optuna
import mlflow
import mlflow.sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

In [9]:
url = 'https://raw.githubusercontent.com/adityasahusomu/Youtube_Comment_Analyzer/refs/heads/main/cleaned_reddit_dataset.csv'
df = pd.read_csv(url).dropna(subset=['clean_comment'])
df.shape

(36662, 2)

In [10]:
df['category'] = df['category'].map({-1: 2, 0: 0, 1: 1})          # Remapping the class labels from [-1, 0, 1] to [2, 0, 1]

df = df.dropna(subset=['category'])          # Removing rows where the target labels (category) are NaN

ngram_range = (1, 3)
max_features = 10000

X_train, X_test, y_train, y_test = train_test_split(df['clean_comment'], df['category'], test_size=0.2, random_state=42, stratify=df['category'])

vectorizer = TfidfVectorizer(ngram_range=ngram_range, max_features=max_features)
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

smote = SMOTE(random_state=42)
X_train_vec, y_train = smote.fit_resample(X_train_vec, y_train)

def log_mlflow(model_name, model, X_train, X_test, y_train, y_test):
    with mlflow.start_run():
        mlflow.set_tag("mlflow.runName", f"{model_name}_SMOTE_TFIDF_Trigrams")
        mlflow.set_tag("experiment_type", "algorithm_comparison")

        mlflow.log_param("algo_name", model_name)

        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        # Logging the model
        mlflow.sklearn.log_model(model, f"{model_name}_model")

# Optuna objective function for XGBoost (Bayesian Optimization)
def objective_xgboost(trial):
    n_estimators = trial.suggest_int('n_estimators', 50, 300)
    learning_rate = trial.suggest_float('learning_rate', 1e-4, 1e-1, log=True)
    max_depth = trial.suggest_int('max_depth', 3, 10)

    model = XGBClassifier(n_estimators=n_estimators, learning_rate=learning_rate, max_depth=max_depth, random_state=42)
    model.fit(X_train_vec, y_train)
    y_pred = model.predict(X_test_vec)
    return accuracy_score(y_test, y_pred)

# Running Optuna for XGBoost, log the best model only
def run_optuna_experiment():
    study = optuna.create_study(direction="maximize")
    study.optimize(objective_xgboost, n_trials=30)

    best_params = study.best_params                                # Getting the best parameters and log only the best model
    best_model = XGBClassifier(n_estimators=best_params['n_estimators'], learning_rate=best_params['learning_rate'], max_depth=best_params['max_depth'], random_state=42)

    log_mlflow("XGBoost", best_model, X_train_vec, X_test_vec, y_train, y_test)                   # Logging the best model with MLflow, passing the algo_name as "xgboost"

run_optuna_experiment()


[I 2025-10-16 10:27:18,384] A new study created in memory with name: no-name-f98e35c7-a5bd-436b-be89-8ead2374d837
[I 2025-10-16 10:30:57,345] Trial 0 finished with value: 0.5330696849856812 and parameters: {'n_estimators': 289, 'learning_rate': 0.0001408489981775656, 'max_depth': 4}. Best is trial 0 with value: 0.5330696849856812.
[I 2025-10-16 10:47:40,709] Trial 1 finished with value: 0.7200327287603983 and parameters: {'n_estimators': 289, 'learning_rate': 0.015110837855735013, 'max_depth': 10}. Best is trial 1 with value: 0.7200327287603983.
[I 2025-10-16 10:51:01,724] Trial 2 finished with value: 0.6780308195827083 and parameters: {'n_estimators': 288, 'learning_rate': 0.019447554955243865, 'max_depth': 4}. Best is trial 1 with value: 0.7200327287603983.
[I 2025-10-16 11:00:32,856] Trial 3 finished with value: 0.6687576708032184 and parameters: {'n_estimators': 272, 'learning_rate': 0.009025185264767987, 'max_depth': 7}. Best is trial 1 with value: 0.7200327287603983.
[I 2025-10-1

🏃 View run XGBoost_SMOTE_TFIDF_Trigrams at: http://ec2-3-15-32-230.us-east-2.compute.amazonaws.com:5000/#/experiments/716770175548013518/runs/9c340999249c496fa2af78dfa683c12c
🧪 View experiment at: http://ec2-3-15-32-230.us-east-2.compute.amazonaws.com:5000/#/experiments/716770175548013518
